In [2]:
import pandas as pd
import gzip
import json
QRELS = ['qrels.trec-covid-additional-judgments.tsv', 'qrels.trec-covid-v1.tsv', 'qrels.trec-covid-v2.tsv']

CORPORA = ['trec-covid-beir/corpus-ORIG.jsonl']

def load_qrels(qrel_file):
    return pd.read_csv('../unprocessed/topics-and-qrels/' + qrel_file, sep='\t')

def load_all_docs():
    ret = []
    for qrel in QRELS:
        ret += list(load_qrels(qrel)['doc_id'])
    return set(ret)

judged_docs = load_all_docs()


In [31]:
!ls -lha trec-covid-v2/corpus.jsonl.gz

-rw-r--r-- 1 maik maik 16M Sep 19 16:19 trec-covid-v2/corpus.jsonl.gz


In [34]:
!zcat trec-covid/corpus.jsonl.gz|head -2

{"_id": "2b73a28n", "title": "Role of endothelin-1 in lung disease", "text": "Endothelin-1 (ET-1) is a 21 amino acid peptide with diverse biological activity that has been implicated in numerous diseases. ET-1 is a potent mitogen regulator of smooth muscle tone, and inflammatory mediator that may play a key role in diseases of the airways, pulmonary circulation, and inflammatory lung diseases, both acute and chronic. This review will focus on the biology of ET-1 and its role in lung disease.", "metadata": {"url": "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC59574/", "pubmed_id": "11686871"}}
{"_id": "zjufx4fo", "title": "Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis", "text": "Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5\u2032 end of the genome fused to different sequences (\u2018bodies\u2019) derived from the 3\u2032 end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synt

In [5]:
translation = {}
current_id = 0

with open('/mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/collection-ORIG.tsv', 'r') as inp, open('/mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/collection.tsv', 'w') as out:
    for l in inp:
        l =  l.split('\t')
        assert len(l) == 2
        
        doc_id = l[0].strip()
        doc_text = l[1].strip()
        if doc_id in judged_docs:
            out.write(str(current_id) + '\t' + doc_text + '\n')
            translation[current_id] = doc_id
            current_id += 1

json.dump(translation, open('/mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/document-ids.json', 'w'))
        

In [6]:
len(judged_docs)

36016

In [8]:
with open('/mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/document-ids.tsv', 'w') as out:
    for k, v in translation.items():
        out.write(f'{v}\t{k}\n')

In [9]:
!sync

In [7]:
translation

{0: '2b73a28n',
 1: 'zjufx4fo',
 2: '8zchiykl',
 3: '5tkvsudh',
 4: 'tvxpckxo',
 5: '6iu1dtyl',
 6: '4k8f7ou1',
 7: 'wnnsmx60',
 8: 'bbvxu8op',
 9: '4cvy9u28',
 10: 'ajlctjeb',
 11: 'zwbc7nnn',
 12: 'vw8xjo9t',
 13: 'i5fcedbo',
 14: 'bjjft7ut',
 15: 'urk7fe34',
 16: 'u09rn09u',
 17: 'kvztcwu2',
 18: 'hwlvk68z',
 19: 'bpnfupzv',
 20: '5eqdrd52',
 21: 'p5jtwb3l',
 22: '1n0rg5vd',
 23: 'nzh87aux',
 24: 'nzk8m912',
 25: 'm56agj4z',
 26: 'byp2eqhd',
 27: 's4y6uxsb',
 28: '9zmyojbu',
 29: '63bos83o',
 30: 'hqc7u9w3',
 31: 'n9sih438',
 32: 'be8rxglx',
 33: '1n69h3i3',
 34: '9zm4per4',
 35: 'wnuqe66q',
 36: 'y2nhss9u',
 37: 'gaemgm0t',
 38: 'kfwbqp4p',
 39: 'mzn448zk',
 40: 'x7wva1ax',
 41: 'zurd5d64',
 42: 'fj9l8che',
 43: 'gnnd0zmo',
 44: '543aq9dx',
 45: 'rzzsmuoc',
 46: 'iy4c7404',
 47: '3ib99gup',
 48: 'gy2b7of9',
 49: 'umvrwgaw',
 50: 'kamvao4b',
 51: '4z38v9rg',
 52: '7gmtp6km',
 53: '4kpkhz0o',
 54: 'fpsyem7x',
 55: 'seass3p0',
 56: 'oz823tw4',
 57: 'sasijnks',
 58: 'slxdonqi',
 59: 'u

In [53]:
!cat /mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/collection.tsv|wc -l

36016


In [56]:
!ls -lh /mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/

total 230M
-r--r--r-- 1 maik webis-users 185M Sep 19 18:56 collection-ORIG.tsv
-r--r--r-- 1 9151 webis-users  45M Sep 19 18:59 collection.tsv
-rw-rw-r-- 1 9151 webis-users 3.7K Sep 19 18:42 queries.tsv


In [45]:
!cp /mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/collection.tsv /mnt/ceph/storage-rw/data-tmp/current/kibi9872/beir-ColBERT/colbert-datasets/trec-covid-beir/collection-ORIG.tsv